<a href="https://colab.research.google.com/github/ariegever/ImageProcessing_Project/blob/main/1_unet_making_assest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This script prepares your Sentinel-1 and Sentinel-2 data in Earth Engine.
# Run this script first to create the assets you will use in the next script.

import ee
import geemap # This library makes GEE in Python much easier
import json
import os

print("Authenticating and Initializing Earth Engine...")
# Trigger GEE authentication
ee.Authenticate()
ee.Initialize(project='ivory-scion-476708-t5') # <-- IMPORTANT: Put your GEE project ID here
print("Earth Engine Initialized.")

# --- 1. USER INPUT ---
# Fill in these variables with your asset paths and preferences

# 1.1: Your Land Cover (LC) asset ID
# This is the full path to the LC map you already uploaded.
LC_ASSET_ID = "projects/ivory-scion-476708-t5/assets/lc_map" # <-- CHANGE THIS

# 1.2: Your GEE asset folder for EXPORTS
# Where do you want to save the new S1 and S2 images?
# Example: "users/YOUR_USERNAME/my_data_assets"
ASSET_EXPORT_FOLDER = "projects/ivory-scion-476708-t5/assets/" # <-- CHANGE THIS

# 1.3: A base name for your new assets
AOI_NAME = "east_negev" # <-- CHANGE THIS (e.g., "my_study_area")

# 1.4: Date range for imagery
START_DATE = '2020-01-01'
END_DATE = '2020-12-31'
year = START_DATE[:4]
print(f"Year: {year}")

# --- 2. LOAD SOURCE ASSETS ---
print(f"Loading Land Cover asset: {LC_ASSET_ID}")
try:
    lc_image = ee.Image(LC_ASSET_ID)
except ee.EEException as e:
    print(f"Error loading LC Asset ID: {e}")
    print("Please double-check your LC_ASSET_ID path.")
    # Stop the script if the main asset can't be loaded.
    raise

# Get the geometry, CRS, and scale from your LC map
# All new imagery will be matched to this.
geometry = lc_image.geometry()
crs = lc_image.projection().crs()
scale = lc_image.projection().nominalScale()

print(f"LC asset loaded. CRS: {crs.getInfo()}, Scale: {scale.getInfo()}m")
print("Target geometry (bounding box):")
print(geometry.bounds().getInfo()['coordinates'])

# --- 3. PROCESS SENTINEL-2 ---

# Cloud masking function for Sentinel-2 SR
def maskS2clouds(image):
    scl = image.select('SCL')
    # Keep good pixels: 4 (veg), 5 (bare), 6 (water), 7 (unclassified), 11 (snow)
    good_qa = ee.List([4, 5, 6, 7, 11])
    mask = scl.remap(good_qa, ee.List.repeat(1, good_qa.length()), 0)

    # Select only the optical bands (B1-B12) for output and apply scaling
    optical_bands = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']).divide(10000)

    return optical_bands.updateMask(mask).copyProperties(image, ["system:time_start"])

print("\nProcessing Sentinel-2 imagery...")
# Load S2, filter, mask clouds, and create a median composite
s2_image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterBounds(geometry)
            .filterDate(START_DATE, END_DATE)
            .map(maskS2clouds)
            .median()
            .clip(geometry)
            )

print("Sentinel-2 processing complete.")

# --- 4. PROCESS SENTINEL-1 ---
print("\nProcessing Sentinel-1 imagery...")
# Load S1, filter, and create a median composite
s1_image = (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(geometry)
            .filterDate(START_DATE, END_DATE)
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .select(['VV', 'VH'])
            .median()
            .clip(geometry)
           )
# --- 5. EXPORT ASSETS ---
print("\nStarting asset export tasks...")

# to prevent 'assets//' error.
if ASSET_EXPORT_FOLDER.endswith('/'):
    ASSET_EXPORT_FOLDER = ASSET_EXPORT_FOLDER[:-1]
    print(f"Removed trailing slash. Export folder: {ASSET_EXPORT_FOLDER}")

# Create the asset folder if it doesn't exist (via geemap)
try:
    geemap.create_ee_folder(ASSET_EXPORT_FOLDER)
    print(f"Ensured asset folder exists: {ASSET_EXPORT_FOLDER}")
except Exception as e:
    print(f"Could not create/verify asset folder: {e}. Assuming it exists.")

# Define full asset IDs for export
s2_asset_id = f'{ASSET_EXPORT_FOLDER}/{AOI_NAME}_Sentinel2_{year}'
s1_asset_id = f'{ASSET_EXPORT_FOLDER}/{AOI_NAME}_Sentinel1_{year}'

# Create and start the export tasks
task_s2 = ee.batch.Export.image.toAsset(
    image=s2_image,
    description=f'Export_{AOI_NAME}_Sentinel2_{year}',
    assetId=s2_asset_id,
    region=geometry,
    scale=scale,
    crs=crs,
    maxPixels=1e13
)
task_s2.start()

task_s1 = ee.batch.Export.image.toAsset(
    image=s1_image,
    description=f'Export_{AOI_NAME}_Sentinel1_{year}',
    assetId=s1_asset_id,
    region=geometry,
    scale=scale,
    crs=crs,
    maxPixels=1e13
)
task_s1.start()

print("\n--- TASKS STARTED ---")
print("Check the 'Tasks' tab in your Google Earth Engine Code Editor.")
print("Wait for the tasks to complete before running the next script.")
print("\nOnce tasks are complete, use these Asset IDs in the next script:")
print(f"LC_ASSET_ID: {LC_ASSET_ID}")
print(f"S2_ASSET_ID: {s2_asset_id}")
print(f"S1_ASSET_ID: {s1_asset_id}")
print("\n--- TASKS STARTED ---")

Authenticating and Initializing Earth Engine...
Earth Engine Initialized.
Year: 2020
Loading Land Cover asset: projects/ivory-scion-476708-t5/assets/lc_map
LC asset loaded. CRS: EPSG:2039, Scale: 25m
Target geometry (bounding box):
[[[34.80282354223304, 31.04993958469053], [35.192004113755004, 31.04993958469053], [35.192004113755004, 31.359692825971788], [34.80282354223304, 31.359692825971788], [34.80282354223304, 31.04993958469053]]]

Processing Sentinel-2 imagery...
Sentinel-2 processing complete.

Processing Sentinel-1 imagery...

Starting asset export tasks...
Removed trailing slash. Export folder: projects/ivory-scion-476708-t5/assets
Could not create/verify asset folder: module 'geemap' has no attribute 'create_ee_folder'. Assuming it exists.

--- TASKS STARTED ---
Check the 'Tasks' tab in your Google Earth Engine Code Editor.
Wait for the tasks to complete before running the next script.

Once tasks are complete, use these Asset IDs in the next script:
LC_ASSET_ID: projects/ivor